### Blood pressure vs Sleep duration
file : abpm_d2.csv
1. subtract the csv file by the columns which I am interested
project 1: blood pressure vs sleep duration
project 2: blood pressure (H/M/L) vs sleep duration

Project 1:
Calculate the mean of blood pressure during sleep and awake.
Do statistical analysis to check whether there is signigficant betweeen two means

Psedocode:
1. subset columns
2. adjust time to HH
3. split to two group (day(work) and night(sleep))
4. calculate the mean blood pressure




Project 2:
First spit the data to three groups (H/M/L diet)
The follow the same steps as Project 1

In [78]:
# import packages, and subset the column I am interested in, save as a variable df_ver1

import pandas as pd; import numpy as np; from scipy import stats

df = pd.read_csv('abpm_d2.csv')
df_ver1 = df[['ID_REL','SBP','DBP','TIME','SODIUM','SLEEPFRO','SLEEPTO']]

In [79]:
# adjust the time from HH:MM to HH
def timechange(t):
    (m,s) = t.split(':')
    result = int(m) 
    return result

# Adjust the time, replace the value in the column
df_ver1['TIME'] = df_ver1['TIME'].apply(timechange)
df_ver1['SLEEPFRO'] = df_ver1['SLEEPFRO'].apply(timechange)
df_ver1['SLEEPTO'] = df_ver1['SLEEPTO'].apply(timechange)

for x in range(len(df_ver1['SLEEPTO'])):
    df_ver1['SLEEPTO'][x] = df_ver1['SLEEPTO'][x] + df_ver1['SLEEPFRO'][x]

#save table as csv file
abpm_project1 = df_ver1.to_csv('abpm_project1.csv')

/home/urameshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/urameshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/urameshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [81]:
# if the time >= sleepfro time but < sleepto, than it is classified as sleeping group
# if the time < sleepfro time but >= sleepto, than it is classified as working group
# subset the table to sleeping and working group
df_ver1.head()
sleeping = []
working = []

for x in range(len(df_ver1['SLEEPTO'])):
    if df_ver1['TIME'][x] >= df_ver1['SLEEPFRO'][x] and df_ver1['TIME'][x] < df_ver1['SLEEPTO'][x]:
        sleeping.append(x)
    else:
        working.append(x)

df_ver1.loc[sleeping].to_csv('sleeping_bp.csv')
df_ver1.loc[working].to_csv('working_bp.csv')
        

In [85]:
# assign variable for sleeping_bp.csv and working_bp.csv
df_sleep = pd.read_csv('sleeping_bp.csv')
df_work = pd.read_csv('working_bp.csv')

print(df_sleep.describe())
print(df_work.describe())

         Unnamed: 0          SBP          DBP         TIME     SLEEPFRO  \
count   6046.000000  6046.000000  6046.000000  6046.000000  6046.000000   
mean   24556.294079   120.279358    72.756864    12.199802    10.118095   
std    14086.405111    16.194431    12.698466     9.392733    10.326784   
min        4.000000    71.000000    36.000000     0.000000     0.000000   
25%    12361.250000   109.000000    64.000000     3.000000     0.000000   
50%    25086.500000   118.000000    72.000000     8.000000     3.000000   
75%    36558.750000   130.000000    81.000000    22.000000    22.000000   
max    48032.000000   209.000000   130.000000    23.000000    23.000000   

           SLEEPTO  
count  6046.000000  
mean     17.068806  
std      10.275618  
min       2.000000  
25%       7.000000  
50%      14.000000  
75%      28.000000  
max      43.000000  
         Unnamed: 0           SBP           DBP          TIME      SLEEPFRO  \
count  42017.000000  42017.000000  42017.000000  42017.0

In [110]:
# Calculate the mean of sbp and dbp of two groups for each ID for same size of data.
'''
1. set group as ID and make new dataframe

'''

id_list = []
for x in df_sleep['ID_REL'].drop_duplicates():
    id_list.append(x)

    
sleep_id_index = []
for x in range(len(df_sleep['ID_REL'])-1):
    if df_sleep['ID_REL'][x] != df_sleep['ID_REL'][x+1]:
        sleep_id_index.append(x+1)

        
work_id_index = []
for x in range(len(df_work['ID_REL'])-1):
    if df_work['ID_REL'][x] != df_work['ID_REL'][x+1]:
        work_id_index.append(x+1)
        
# check whether the number of id_index is the same
len(work_id_index) == len(sleep_id_index)  #True

# add zero at the begining of the list
work_id_index.insert(0,0) ; sleep_id_index.insert(0,0)

True

In [202]:
# Calculate the mean of blood pressure (SBP,DBP) from work and sleep
sleep_sbp = []; sleep_dbp = []; work_sbp = [] ; work_dbp = []

for x in range(len(id_list)):
    sleep_sbp.append(df_sleep['SBP'][sleep_id_index[x]:sleep_id_index[x]+1].mean())
    sleep_dbp.append(df_sleep['DBP'][sleep_id_index[x]:sleep_id_index[x]+1].mean())
    work_sbp.append(df_work['SBP'][work_id_index[x]:work_id_index[x]+1].mean())
    work_dbp.append(df_work['DBP'][work_id_index[x]:work_id_index[x]+1].mean())

# Combine the result of the previous step with ID to form dictionary    
sleep_sbp_dict = {} ; sleep_dbp_dict = {} 

for y in range(len(id_list)):
    sleep_sbp_dict[id_list[y]] = sleep_sbp[y]
    sleep_dbp_dict[id_list[y]] = sleep_dbp[y]
    
    
# Use dictionaries to Create DataFrame, and add columns to make paired data for t-test
   # sbp_dash is the dataframe of paired data shows the sbp from working and sleeping condition
sbp_dash = pd.DataFrame.from_dict(pd.DataFrame.from_dict(sleep_sbp_dict, orient='index',columns=['sleeping_sbp']))
sbp_dash['working_sbp'] = work_sbp

   # dbp_dash is the dataframe of paired data shows the dbp from working and sleeping condition
dbp_dash = pd.DataFrame.from_dict(pd.DataFrame.from_dict(sleep_dbp_dict, orient='index',columns=['sleeping_dbp']))
dbp_dash['working_dbp'] = work_dbp

# Save the file as csv for later-use if needed.
sbp_dash.to_csv('mean_sbp_dash.csv')
dbp_dash.to_csv('mean_dbp_dash.csv')


In [206]:
'''
Paired sample t-test (correlated pairs t-test, dependent samples t test) 
> compares means from the same group at different times (say, one year apart)

1. Calculate each IDs means of sbp and dbp (working and sleeping)
2. Subtraction the mean during working and sleeping
3. Statistically analysis by using paired sample t-test

H0: Null hypothesis: Blood Pressure of sleep and work are the same
H1: Alternate Hypothesis: Blood Pressure of sleep and work are different

'''
# Perform t-test with input list work_dbp, sleep_dbp, work_sbp, sleep_sbp

ttest_dbp = stats.ttest_ind(work_dbp,sleep_dbp)
matrix_ttest_dbp = [['', 'Test Statistic', 'p-value'],
                    ['dbp_data', ttest_dbp[0], ttest_dbp[1]]]

ttest_sbp = stats.ttest_ind(work_sbp, sleep_dbp)
matrix_ttest_sbp = [['', 'Test Statistic', 'p-value'],
                   ['sbp_data', ttest_sbp[0], ttest_sbp[1]]]

print (matrix_ttest_dbp,matrix_ttest_sbp)
'''
matrix_ttest_dbp:
[['', 'Test Statistic', 'p-value'], 
['dbp_data', 15.374656533292148, 4.704674945582392e-46]] 

matrix_ttest_sbp
[['', 'Test Statistic', 'p-value'], 
['sbp_data', 61.23685649207045, 2.90169113078851e-280]]
'''

[['', 'Test Statistic', 'p-value'], ['dbp_data', 15.374656533292148, 4.704674945582392e-46]] [['', 'Test Statistic', 'p-value'], ['sbp_data', 61.23685649207045, 2.90169113078851e-280]]
